In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import glob
import os

In [165]:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

for year in years:
    base = 'http://profiles.doe.mass.edu/state_report/enrollmentbyracegender.aspx?mode=school&year={}&Continue=View+Report&export_excel=yes'
    base = base.format(year)
    print(base)
    filename = "student_"+"raceeth_"+str(year)+".xls"
    with open(filename, 'w') as output:
        output.write(requests.get(base).text)

http://profiles.doe.mass.edu/state_report/enrollmentbyracegender.aspx?mode=school&year=2017&Continue=View+Report&export_excel=yes


In [ ]:
## Create function to clean up DFs

In [16]:
def clean_file(file):
    df = pd.read_html(file)
    df = pd.DataFrame(df[0])
    header = df.iloc[0]
    df.columns = header
    df = df[1:]
    file_name = os.path.splitext(file)[0]
    df["Source"] = file_name
    df['Year'] = df["Source"].str[-4:]
    return df

In [ ]:
# Create master CSV

In [17]:
all_files = glob.glob("student_raceeth_201[0-5].xls")
df_list = []

for file in all_files:
    file = clean_file(file)
    df_list.append(file)
    
df = pd.concat(df_list)

In [ ]:
df = df.rename(columns={'ORG CODE': 'Org Code'})
df

In [ ]:
# Test out mask

In [ ]:
mask = df["Org Code"] == "04450105"
df_abbyk = df[mask]

df_abbyk

In [ ]:
# Set ORG CODE as index and write master file to CSV

In [20]:
df = df.set_index('Org Code')
df.to_csv("student_raceeth_all.csv")